<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Generate_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

     |████████████████████████████████| 39.9MB 74kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 665kB 58.0MB/s 
     |████████████████████████████████| 22.2MB 99kB/s 
     |████████████████████████████████| 3.3MB 59.2MB/s 
     |████████████████████████████████| 204kB 60.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [2]:
#Clone git
!git clone -b before_framestacking https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 663 (delta 124), reused 3 (delta 0), pack-reused 460
Receiving objects: 100% (663/663), 17.36 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (407/407), done.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# TESTING

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


In [6]:
# run_experiment(2, levels=50, save_interval=5e5)

In [7]:
# #for Experiment 7
# file_name = "Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_13516800steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [8]:
# #For Experiment 2
# file_name = "Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_13516800steps"
# set_hyperparameters(baseline="PPO")
# h.value_clipping = True
# h.num_levels = 50
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [9]:
# #for Experiment 8
# file_name = "Experiment8_50levels_Run_05Dec_08h34m02s_4005888steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# h.death_penalty = True
# h.penalty = 5
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

# Generate video

In [10]:
from utils import make_env
import imageio

def old_policy_evaluation(model, video_name=None, print_output=True, test_on_training=False, off_set = 0):
  policy = model.policy

  if video_name is None:
    video_name = model.file_name
  # Make evaluation environment
  if test_on_training:
    eval_env = make_env(model.num_envs, num_levels=model.num_levels)
    video_name += "_TEST_ON_TRAINING"
  else:
    eval_env = make_env(model.num_envs, start_level=model.num_levels+off_set, num_levels=model.num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []

  # Evaluate policy
  policy.eval()
  # for _ in range(512):  #<--- Hardcoded batch size?
  while True:
    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)
    if done[0]:
      break

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

## New code

In [25]:
from utils import make_env
import imageio

VIDEO_PATH = '/content/drive/My Drive/02456-Deep-Learning-Project/Videos/'

def generate_videos(model, video_name=None, print_output=True, test_on_training=False, off_set = 0, nr_of_levels=5, save_video=False, dist_mode="easy"):
  policy = model.policy

  if video_name is None:
    video_name = model.file_name
  
  # Make evaluation environment
  if test_on_training:
    eval_env = make_env(model.num_envs, num_levels=model.num_levels, normalize_reward = False, dist_mode=dist_mode)
    video_name += "_TEST_ON_TRAINING"
  else:
    eval_env = make_env(model.num_envs, start_level=model.num_levels+off_set, num_levels=model.num_levels, normalize_reward = False, dist_mode=dist_mode)
  obs = eval_env.reset()

  frames = []
  total_reward = []
  level_counter=1
  # Evaluate policy
  policy.eval()

  while True:
    
    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)
    if done[0]: #next level
      #Calculate level reward
      level_reward = torch.stack(total_reward).sum(0)[0]
      
      if save_video:
        # Save frames as video
        frames = torch.stack(frames).cpu().numpy()
        name = video_name+'_level{:d}_reward{:.0f}.mp4'.format(level_counter, level_reward)
        imageio.mimsave(VIDEO_PATH + name, frames, fps=25)
        print("Saved video: {}".format(name))
      else:
        print(level_counter)
        print(level_reward)
      
      #check if all levels done
      level_counter +=1
      if level_counter > nr_of_levels:
        break #done testing

      #reset variables for next level
      frames=[]
      total_reward = []

## Load model

## Impala DP 5

In [20]:
# #for Experiment 8
# file_name = "Experiment8_200levels_Run_06Dec_16h17m59s_loaded_08Dec_13h28m10s_8028160steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 200
# h.batch_size=512
# h.death_penalty = True
# h.penalty = 5
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# policy = model.load_policy(file_name)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15
Loaded current model from models folder with name Experiment8_200levels_Run_06Dec_16h17m59s_loaded_08Dec_13h28m10s_8028160steps.pt


In [13]:
# old_policy_evaluation(model, test_on_training=True)

In [26]:
# generate_videos(model, video_name="Final_Impala_dp5_200levels_test10", save_video=True, nr_of_levels=10)

Saved video: Final_Impala_dp5_200levels_test10_level1_reward20.mp4
Saved video: Final_Impala_dp5_200levels_test10_level2_reward19.mp4
Saved video: Final_Impala_dp5_200levels_test10_level3_reward8.mp4
Saved video: Final_Impala_dp5_200levels_test10_level4_reward14.mp4
Saved video: Final_Impala_dp5_200levels_test10_level5_reward44.mp4
Saved video: Final_Impala_dp5_200levels_test10_level6_reward54.mp4
Saved video: Final_Impala_dp5_200levels_test10_level7_reward61.mp4
Saved video: Final_Impala_dp5_200levels_test10_level8_reward62.mp4
Saved video: Final_Impala_dp5_200levels_test10_level9_reward24.mp4
Saved video: Final_Impala_dp5_200levels_test10_level10_reward16.mp4


In [24]:
# generate_videos(model, video_name="Final_Impala_dp5_200levels_test10", save_video=True, nr_of_levels=10, test_on_training=True)

Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level1_reward25.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level2_reward7.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level3_reward54.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level4_reward53.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level5_reward6.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level6_reward9.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level7_reward48.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level8_reward51.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level9_reward46.mp4
Saved video: Final_Impala_dp5_200levels_test10_TEST_ON_TRAINING_level10_reward20.mp4


### Impala no penalty 200 levels

In [27]:
#for Experiment 7
file_name = "Experiment7_200levels_Run_06Dec_16h18m31s_loaded_08Dec_14h21m55s_8019968steps"
set_hyperparameters(baseline="Impala")
h.value_clipping = True
h.num_levels = 200
h.batch_size=512
model = PPO(print_output=True, eval=True, save_interval=5e5)
model.load_policy(file_name)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15
Loaded current model from models folder with name Experiment7_200levels_Run_06Dec_16h18m31s_loaded_08Dec_14h21m55s_8019968steps.pt


Policy(
  (encoder): ImpalaEncoder(
    (block1): ImpalaBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (res1): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (res2): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (block2): ImpalaBlock(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (res1): ResidualBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (res2): ResidualBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(

In [ ]:
generate_videos(model, video_name="Final_Impala_200levels_test10", save_video=True, nr_of_levels=10)

Saved video: Final_Impala_200levels_test10_level1_reward9.mp4
Saved video: Final_Impala_200levels_test10_level2_reward15.mp4
Saved video: Final_Impala_200levels_test10_level3_reward20.mp4


In [ ]:
generate_videos(model, video_name="Final_Impala_200levels_test10", save_video=True, nr_of_levels=10, test_on_training=True)

In [16]:
# old_policy_evaluation(model)

### PPO 200 levels

In [17]:
# #For Experiment 2
# file_name = "Experiment2_200levels_Run_07Dec_13h49m13s_loaded_08Dec_00h03m23s_7012352steps"
# set_hyperparameters(baseline="PPO")
# h.value_clipping = True
# h.num_levels = 200
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# policy=model.load_policy(file_name)

In [18]:
# for i in range(4):
#   result=old_policy_evaluation(model, video_name="PPO_200levels_{}".format(i), off_set=i, print_output=False)
#   print("{}: {}".format(i, result))
  

In [19]:
# old_policy_evaluation(model, test_on_training=True)